In [1]:
!pip3 uninstall -y transformers

Found existing installation: transformers 4.28.1
Uninstalling transformers-4.28.1:
  Successfully uninstalled transformers-4.28.1


In [2]:
!pip3 install pdfplumber reportlab transformers sacremoses

  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)


In [3]:
!sudo apt-get install -y wkhtmltopdf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.5-1build1).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.



- [翻訳したテキストのフォント](https://forest.watch.impress.co.jp/library/software/ipaexfont/download_10822.html)

In [4]:
import os
from tqdm.notebook import tqdm
tqdm.pandas()
import glob

import pdfplumber
from transformers import MarianMTModel, MarianTokenizer
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import re

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-15 06:29:36.838443: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
input_path = './input'
file_path_list = glob.glob(input_path + '/*.pdf')
file_path_list

['./input/2111.13462v1.pdf']

In [6]:
input_pdf_path = file_path_list[0]
output_pdf_path = './output' + file_path_list[0].split('/')[-1]
os.makedirs('./output', exist_ok=True)

In [7]:
input_path = './input'
file_path_list = glob.glob(input_path + '/*.pdf')
file_path_list

['./input/2111.13462v1.pdf']

In [8]:
input_pdf_path = file_path_list[0]
output_pdf_path = './output/' + file_path_list[0].split('/')[-1]

In [9]:
def extract_pdf_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in tqdm(pdf.pages):
            text += page.extract_text()
    return text

In [10]:
import re
import string

def remove_stopwords(text):
    # 改行文字やタブ文字を除去します
    text = text.replace('\n', ' ').replace('\t', ' ')
    # Unicodeエスケープシーケンスを除去します
    text = re.sub(r'\\[xX][a-fA-F0-9]{2}', '', text)
    # パンクチュエーション文字を除去します
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [11]:
def split_sentences(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return sentences

In [12]:
from transformers import pipeline

# Load the MarianMTModel and MarianTokenizer for English to Japanese translation
model_name = 'staka/fugumt-en-ja'
fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')

def translate_text(text, model_name):
    # tokenizer = MarianTokenizer.from_pretrained(model_name)
    # model = MarianMTModel.from_pretrained(model_name)

    sentences = split_sentences(text)
    translated_sentences = []

    for sentence in tqdm(sentences):
        # tokenized_text = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
        # translated_tokens = model.generate(**tokenized_text, max_length=512, num_return_sequences=1)
        # translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        
        try:
            translated_sentence = fugu_translator(sentences)
        except IndexError:
            pass
        
        japanese_translations = []
        for translation in translated_sentence:
            japanese_translations.append(translation['translation_text'])
        translated_sentence = ''.join(japanese_translations)
        
        translated_sentences.append(translated_sentence)

    translated_text = "\n".join(translated_sentences)
    return translated_text

In [13]:
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

def save_text_to_pdf(output_path, translated_text, font_path="ipaexm.ttf"):
    pdfmetrics.registerFont(TTFont("IPAexGothic", font_path))
    
    c = canvas.Canvas(output_path, pagesize=letter)
    c.setFont("IPAexGothic", 14)
    x, y = 30, 750
    for line in translated_text.split("\n"):
        c.drawString(x, y, line)
        y -= 20
        if y < 20:
            c.showPage()
            y = 750
    c.save()

In [14]:
# PDFからテキストを抽出
text = extract_pdf_text(input_pdf_path)

  0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
# text = remove_stopwords(text)

In [16]:
# テキストを日本語に翻訳
model_name = "staka/fugumt-en-ja" # 英語から日本語への翻訳モデル
translated_text = translate_text(text, model_name)

  0%|          | 0/255 [00:00<?, ?it/s]

In [17]:
translated_text[:50]

'Log Data Thorsten Wittkopp, Philipp Wiesner, Domin'

In [18]:
# 翻訳されたテキストをPDFとして保存
os.makedirs('./output', exist_ok=True)
save_text_to_pdf(output_pdf_path, translated_text)

In [19]:
# 翻訳されたテキストをtxtとして保存
def save_text_to_txt(output_path, translated_text):
    with open(output_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(translated_text)

In [20]:
output_txt_path = output_pdf_path.split('.')[1]
output_txt_path = '.' + output_txt_path + '.txt'
save_text_to_txt(output_txt_path, translated_text)